In [8]:
import argparse
import torch 
import torch.nn as nn
import numpy as np
import h5py
from torchsummary import summary
import load_data
from torch.utils.tensorboard import SummaryWriter

HyperParameters

In [9]:
parser = argparse.ArgumentParser()
parser.add_argument("--n_epochs", type=int, default=50, help="number of epochs of training")
parser.add_argument("--batch_size", type=int, default=50, help="size of the batches")

parser.add_argument("--n_dat", type=int, default=9000, help="number of used data")
parser.add_argument("--n_val", type=int, default=50, help="number of test data")
parser.add_argument("--lr", type=float, default=0.0002, help="adam: learning rate")

parser.add_argument("--b1", type=float, default=0.5, help="adam: decay of first order momentum of gradient")
parser.add_argument("--b2", type=float, default=0.999, help="adam: decay of first order momentum of gradient")
parser.add_argument("--n_cpu", type=int, default=8, help="number of cpu threads to use during batch generation")

parser.add_argument("--latent_dim", type=int, default=100, help="dimensionality of the latent space")


parser.add_argument("--sp_size", type=int, default=500, help="size of each spectra length")

parser.add_argument("--n_pars", type=int, default=5, help="number of each parameters sample")
parser.add_argument("--n_critic", type=int, default=5, help="number of training steps for discriminator per iter")

parser.add_argument("--clip_value", type=float, default=0.01, help="lower and upper clip value for disc. weights")
parser.add_argument("--sample_interval", type=int, default=100, help="interval between spec sampling")
parser.add_argument("--epoch_interval", type=int, default=10, help="interval between epoch ")
parser.add_argument("--epoch_sample", type=int, default=50, help="Random sample for epoch interval")
parser.add_argument("--lambda_l1", type=float, default=10, help="Balance the adversial loss and l1 loss")
parser.add_argument("--lambda_phy", type=float, default=10, help="Physcial loss weight")
opt, _ = parser.parse_known_args()

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if torch.cuda.is_available():
    print("GPU is available")
    print(f"Number of GPUs: {torch.cuda.device_count()}")
    print(f"Current GPU: {torch.cuda.current_device()}")
    print(f"GPU Name: {torch.cuda.get_device_name(torch.cuda.current_device())}")
else:
    print("No GPU available")

No GPU available


In [11]:
writer = SummaryWriter(log_dir="runs/training_logs")

# Data Preprocess

get data

In [12]:
spec,cond = load_data.datavalue(opt.sp_size)

Necessary Shape
spec: (9900, 4556) age (9900,) metallicity (9900,) smass (9900,) z (9900,)
Shape of condition (shape): (9900, 4)
Shape of the low energy bins spectra (9900, 500)


Get Dataloader

In [13]:
train_loader = load_data.prepareDataSet(spec,cond,opt.n_dat,opt.n_val,device)

training data size:  torch.Size([8950, 500]) label:  torch.Size([8950, 4])
validation data size:  torch.Size([50, 500]) label:  torch.Size([50, 4])
test data size:  torch.Size([900, 500]) label:  torch.Size([900, 4])


# Model

## Generator

In [32]:
import ResNet 
from ResNet import restnet18cbam
def conv3x3(in_planes, out_planes, stride=1):
    "3x3 convolution with padding"
    return nn.Conv1d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)
class Generator(nn.Module):
    def __init__(self):
        super(Generator,self).__init__()
        self.numclass = opt.sp_size
        def block(in_feat,out_feat,normalize=True):
            layers = [nn.Linear(in_feat, out_feat)]
            if normalize:
                layers.append(nn.BatchNorm1d(out_feat))
            layers.append(nn.SELU())
            return layers    
        self.model = nn.Sequential(
            *block(opt.latent_dim,opt.latent_dim*2),
            *block(opt.latent_dim*2,opt.sp_size)
        )
        self.preconv = nn.Sequential(
            conv3x3(1,3),
            nn.BatchNorm1d(3),
            nn.ELU()
        ) 
        self.Residual = restnet18cbam(self.numclass,cbam = False,linear=True)
    def forward(self,label,noise):
        x = torch.cat((label,noise),dim=-1)
        print(x.size())
        x = self.model(x)
        x = x.unsqueeze(dim=1)
        x = self.preconv(x)
        x = self.Residual(x)
        return x

## Distriminator

In [33]:
class Discriminator(nn.Module):
    def __int__(self):
        super(Discriminator,self).__init__()
        self.numberclass = 1
        def convblock (inchanel,outchanel,kernel_szie=3,stride=1,padding=1,batchnorm=True):
            layers = [nn.Conv1d(inchanel, outchanel,kernel_szie=kernel_szie,stride=stride,padding=1)]
            if batchnorm:
                layers.append(nn.BatchNorm1d(outchanel))
            layers.append(nn.ELU())
            return layers 
        def linearblock(infeature,outfeature,batchnorm=True):
            layers = [nn.Linear(infeature, outfeature)]
            if batchnorm:
                layers.append(nn.BatchNorm1d(outfeature))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers 
        self.model = nn.Sequential(
            *convblock(inchanel=1,outchanel=3,kernel_szie=1,batchnorm=False),
            *convblock(inchanel=3,outchanel=32),
            *convblock(inchanel=32,outchanel=32)
        )

        self.connection = nn.Sequential(
            *linearblock(20,x)
        )
    def forward(self,label,spec):
        x = nn.cat((label,spec),dim=-1)
        x = x.unsqueeze(dim=1)
        x = self.model(x)
        x = torch.flatten(x, 1) 
        x = self.connection(x)
        return x 


# Train

In [34]:
generator = Generator()
discriminator = Discriminator()
generator.to(device)
discriminator.to(device)

Discriminator()

In [35]:
summary(generator, [(4,),(opt.latent_dim,)])
summary(discriminator, [(4,), (opt.sp_size,)])

RuntimeError: mat1 and mat2 shapes cannot be multiplied (2x104 and 100x200)